<h1><bold>
<hr style="border-width:2px;border-color:#1664c8">
Partie <b>4</b><br>
    <center>Identification de langue parmi 95<br></center>
<div style="text-align: right;">Version 4</div><br>
<hr style="border-width:2px;border-color:#1664c8">
</h1></bold>

# **Identification de langue avec :**
>### **- des '*Sparse*' Bag Of Words**
>### **- une Tokenisations Tiktoken**
>### **- CountVectorizer utilisant une tokenisation '*custom*'**
>### **- un Classificateurs Naïve Bayes**

## **1 - Contruction des classificateurs**

#### **Chargement des biblothèques nécéssaires** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tiktoken
import random
import joblib
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Ce parametre permet éventuellement d'équilibrer de nombre de phrase par langue.
# Si ce parametre est très grand, tout le corpus sera lu. 
nb_phrase_lang = 10000000

import warnings
warnings.filterwarnings('ignore')

#### **Lectures des phrases de "sentences-big.csv", et de leur étiquette "Langue" pour les langues sélectionnées**

In [2]:
# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path):
    df = pd.read_csv(path, index_col ='id')
    return df

df = create_lang_df('../data/multilingue/sentences-big.csv')
lan_code = list(set(df['lan_code']))
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
n_rows = len(df)
print('Nombre de lignes de sentence.csv:',n_rows)
print('Nombre de langues à classer:',len(lan_code))
df

Nombre de lignes de sentence.csv: 10345978
Nombre de langues à classer: 404


,lan_code,sentence
0,ber,Yeqber uɛebbuḍ-iw seg wayen ččiɣ.
1,ita,Sono venute alla moschea.
2,heb,אני לא יכול לפענח מה קרה.
3,rus,У меня очень много дел.
4,ita,Andiamo a imparare qualcosa in Uganda.
...,...,...
10345973,deu,Wir werden das Problem nicht aufgreifen.
10345974,fra,Je suis cuit !
10345975,epo,Atomenergio estas sekura.
10345976,tok,o pana ala e moku tawa soweli tomo.


#### **Réalisation d'un jeu de données d'entrainement et de test**

In [3]:
# créer 2 dataframes: 1 train (95% des phrases) et 1 test (5% des phrases)
n_train = int(n_rows*0.95)
df_train = df.iloc[:n_train].sample(frac=1, random_state=42).reset_index(drop=True)
df_test = df.iloc[n_train:].sample(frac=1, random_state=24).reset_index(drop=True)
pd.set_option('display.max_colwidth', 150)
df_lan = pd.DataFrame(data= df.groupby('lan_code').size(), columns = ['nb_phrases_lang'] )

# Filtrage des langues qui ont peu de phrases (>2000)
df_lan = df_lan.loc[df_lan['nb_phrases_lang']>=2000]
list_lan = list(set(df_lan.index))
df_train = df_train[df_train['lan_code'].isin(list_lan)]
df_test = df_test[df_test['lan_code'].isin(list_lan)]
print('df_train:')
display(df_train)
print('Nombre de lignes par langue:')
display(df_lan)


df_train:


,lan_code,sentence
0,deu,"Wie die Erde entstanden ist, das ist eine Frage, die sich allen stellt."
1,deu,Drinnen ist etwas Lebendiges.
2,rus,Я не хотела замуж.
3,kab,Ssarden-ak-t.
4,por,"Estou piscando para ele, mas ele não está olhando."
...,...,...
9828674,tat,"Йөрәген бүләк иткәннәр, кан әйләнешендә кыенлыклар барлыкка килсә, гаҗәпләнергә тиеш түгел."
9828675,ber,Cmumeḥ-d yid-i ma ulac aɣilif.
9828676,kab,Ceɛɛel-as ɣer Tasga Mellul.
9828677,eng,Tom can't stand vegetables.


Nombre de lignes par langue:


,nb_phrases_lang
lan_code,
afr,4137
ara,38651
arq,2336
asm,3205
avk,4102
...,...
war,2025
wuu,4757
yid,9632


#### **Préparation de la vectorisation par CountVectorizer** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [4]:
# Selection du tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

# Les 2 fonctions suivantes sont nécéssaires afin de sérialiser ces parametre de CountVectorizer
# et ainsi de sauvegarder le vectorizer pour un un usage ultérieur sans utiliser X_train pour  le réinitialiser

def custom_tokenizer(text):
    tokens = tokenizer.encode(text)  # Cela divise le texte en mots
    return tokens

def custom_preprocessor(text):
    return text

# CountVectorizer a une liste de phrase en entrée.
# Cette fonction met les données d'entrée dans le bon format
def format_to_vectorize(data):
    X_tok = []
    if "DataFrame" in str(type(data)):sentences = data.tolist()
    elif "str" in str(type(data)):
        sentences =[data]
    else: sentences = data
                          
    for sentence in sentences:
        X_tok.append(sentence) # ('¤'.join([tokenizer.decode([ids]) for ids in tokenizer.encode(sentence)])+'¤')
    return X_tok

#### **Création de la fonction Vectorizer et de la fonction de création d'un Bags  Of Worlds** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [5]:
# Création d'un vectorizer et du sparse BOW (X_train) avec le nombre d'apparitions
global vectorizer, dict_ids, dict_token

def create_vectorizer(X_train_tok):
    global vectorizer, dict_ids, dict_token
    
    # token_pattern = r"[a-zA-Z0-9\s\.\,\?\:\;]+" 
    # vectorizer = CountVectorizer(analyzer="word", tokenizer=lambda x: tokenizer.encode(x), preprocessor=lambda x: x) #,token_pattern=token_pattern
    vectorizer = CountVectorizer(analyzer="word", tokenizer=custom_tokenizer, preprocessor=custom_preprocessor) #,token_pattern=token_pattern
    vectorizer.fit(X_train_tok)
    
    # Création de dictionnaire des Token et des ids 
    dict_token = {tokenizer.decode([cle]): cle for cle, valeur in vectorizer.vocabulary_.items()}
    dict_ids = {cle: tokenizer.decode([cle]) for cle, valeur in vectorizer.vocabulary_.items()} #dict_ids.items()}
    return 

def create_BOW(data, vectorizer_to_create=False):
    global vectorizer
    
    X_tok = format_to_vectorize(data)
    if vectorizer_to_create:
        create_vectorizer(X_tok)
    X = vectorizer.transform(X_tok)
    return X

#### **Création du BOW Train**

In [6]:
X_train = create_BOW(df_train['sentence'], True)
y_train = df_train['lan_code'].values.tolist()

#### **Création du BOW Test**


In [7]:
X_test = create_BOW(df_test['sentence'])
y_test = df_test['lan_code'].values.tolist()

#### **Sauvegarde/Chargement du vectorizer** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [8]:
# Definition de fonction de sauvegarde et chargement du dictionnaire des tokens utilisés
def save_vectorizer(vectorizer):
    path = '../data/vectorizer_tiktoken_big.pkl'
    joblib.dump(vectorizer, path)

def load_vectorizer():
    global dict_token, dict_ids, nb_token
    
    path = '../data/vectorizer_tiktoken_big.pkl'
    vectorizer = joblib.load(path)
    dict_token = {tokenizer.decode([cle]): cle for cle, valeur in vectorizer.vocabulary_.items()}
    dict_ids = {cle: tokenizer.decode([cle]) for cle, valeur in vectorizer.vocabulary_.items()} #dict_ids.items()}
    nb_token = len(vectorizer.vocabulary_)
    return vectorizer

In [9]:
# save_vectorizer(vectorizer)

vectorizer = load_vectorizer()

#### **Definition d'une fonction ids->colonne de X_train et col->ids**

In [10]:
def ids2col(list_ids):
    d = dict(vectorizer.vocabulary_.items())
    list_col = []
    for ids in list_ids:
        list_col.append(d[ids])
    return list_col

def col2ids(list_col):
    d = dict(vectorizer.vocabulary_.items())
    list_ids = []
    for col in list_col:
        for ids, c in d.items():
            if col==c:
                list_ids.append(ids)
                break
    return list_ids

#### **Création d'un dictionnaire des tokens avec leur fréquence d'apparition dans Train**
#### **Définition d'une liste de token trié par fréquence d'apparition**

In [11]:
freq = X_train.sum(axis=0)
list_ids = col2ids(range(len(dict_ids)))
list_token = [tokenizer.decode([ids]) for ids in list_ids]
dict_freq = dict(zip(list_token,freq.tolist()[0]))
dict_freq = dict(sorted(dict_freq.items(), key=lambda x: x[1], reverse=True))

def ids2token(ids):
    for token, valeur in dict_ids.items():
        if valeur == ids:
            token_trouvee = token
            break
    return token_trouvee

nb_token = len(vectorizer.vocabulary_)
print("Nombre de tokens :",nb_token)

# Définition d'une liste 'écrite' des tokens : decoded_keys
# decoded_keys = [tokenizer.decode([ids]) for ids in [ids2token(key) for key in list(dict_freq.keys())]]
decoded_keys = list(dict_freq.keys())
print("Liste des 50 tokens les plus fréquents:",decoded_keys[:50])

# vocab_size = max(max(row) for row in X_train) + 1

Nombre de tokens : 58951
Liste des 50 tokens les plus fréquents: ['.', ',', '?', ' a', 'i', 'Tom', 'a', 'u', ' to', ' la', ' de', ' ', ' t', ' d', 'I', 'as', ' Tom', ' k', 'е', 'en', ' n', 'а', 'и', ' in', ' the', ' y', 'ו�', 'is', 'у', 'o', 'о', 'ом', ' с', 'er', ' в', 't', 'z', 'T', '!', ' that', 'ı', '。', ' i', "'t", 'י�', ' ad', 'in', ' ne', 'em', ' is']


#### **Choix du nom du fichier de sauvegarde du classifieur** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [12]:
def get_file_name(titoken_tokenization, classifier):
    return "id_lang_tiktoken_"+classifier+"_sparse_big.pkl"


#### **Création d'un classificateur avec l'algorithme Naïve Bayes**

In [13]:
from sklearn import naive_bayes

# On definit le classificateur Naive Bayes et on l'entraine sur les données Train:
clf_nb = naive_bayes.MultinomialNB()  # BernoulliNB() # MultinomialNB() 
clf_nb.fit(X_train, y_train)

# Save the model to a file
# joblib.dump(clf_nb, "../data/id_lang_tiktoken_nb_sparse_big.pkl") ######### 

MultinomialNB()

In [14]:
# Chargement du classificateur sauvegardé
#clf_nb = joblib.load("../data/id_lang_tiktoken_nb_sparse_big.pkl")

# Verification de l'efficacité du classificateur grace à la matrice confusion
y_pred = clf_nb.predict(X_test)
accuracy_naive_bayes = accuracy_score(y_test, y_pred)
print("Matrice de confusion du classificateur Naïve Bayes")
ct = pd.crosstab(y_test,y_pred,rownames=['Classe réelle'], colnames=['Classe prédite'])
display(ct)
print("Accuracy Naïve Bayes = {:.3f}".format(accuracy_naive_bayes))

Matrice de confusion du classificateur Naïve Bayes


Classe prédite,afr,ara,arq,asm,avk,aze,bel,ben,ber,bre,...,uig,ukr,urd,vie,vol,war,wuu,yid,yue,zsm
Classe réelle,,,,,,,,,,,,,,,,,,,,,
afr,149,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ara,0,1923,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arq,0,103,0,0,0,0,0,0,2,0,...,1,0,0,0,0,0,0,0,0,0
asm,0,0,0,110,0,0,0,63,0,0,...,0,0,0,0,0,0,0,0,0,0
avk,0,0,0,0,180,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
war,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,73,0,0,0,0
wuu,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,75,0,2,0
yid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,420,0,0


Accuracy Naïve Bayes = 0.960


In [15]:
ct_weurope = pd.DataFrame(ct)
lignes_a_sommer = ct.index.difference(['eng','deu','fra','ita','spa'])
somme_l = ct.loc[lignes_a_sommer].sum()
ct_weurope.loc['etc'] = somme_l
ct_weurope = ct_weurope.drop(index = ct_weurope.index.difference(['eng','deu','fra','ita','spa','etc']))

colonnes_a_sommer = ct_weurope.columns.difference(['eng','deu','fra','ita','spa'])
somme_c = ct_weurope[colonnes_a_sommer].sum(axis=1)
ct_weurope['etc']= somme_c
ct_weurope = ct_weurope.drop(columns=ct_weurope.columns.difference(['eng','deu','fra','ita','spa','etc']))
display(ct_weurope)
accuracy_weurope = (ct_weurope['eng']['eng']+ct_weurope['deu']['deu']+ct_weurope['fra']['fra']+ct_weurope['ita']['ita']+ct_weurope['spa']['spa'])/(ct_weurope.sum().sum()-ct_weurope['etc']['etc'])
print("Accuracy pour les langues d'Europe de l'Ouest = {:.3f}".format(accuracy_weurope))

Classe prédite,deu,eng,fra,ita,spa,etc
Classe réelle,,,,,,
deu,29330,4,1,1,0,39
eng,11,78916,3,7,8,182
fra,8,8,24833,26,13,49
ita,1,2,22,40340,38,173
spa,1,1,4,35,18321,147
etc,85,46,102,332,447,320029


Accuracy pour les langues d'Europe de l'Ouest = 0.991


#### **Definition de fonction identificateur de langue** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [16]:
import json

# Chargement du classificateur sauvegardé
clf_nb = joblib.load("../data/id_lang_tiktoken_nb_sparse_big.pkl")
vectorizer = load_vectorizer()

# Lisez le contenu du fichier JSON
with open('../data/multilingue/lan_to_language.json', 'r') as fichier:
    lan_to_language = json.load(fichier)



def lang_id_nb(sentences):
    if "str" in str(type(sentences)):
        return lan_to_language[clf_nb.predict(create_BOW(sentences))[0]]
    else: return [lan_to_language[l] for l in clf_nb.predict(create_BOW(sentences))]
   

#### **Exemples d'utilisation** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [17]:
# Instanciation d'un exemple
exemples = ["Er weiß überhaupt nichts über dieses Buch.",                                                             # Phrase 0
            "france is often snowy during spring , and it is relaxing in january .",                                  # Phrase 1
           "elle adore les voitures très luxueuses, et toi ?",                                                        # Phrase 2
           "she loves very luxurious cars, don't you?",                                                               # Phrase 3
           "vamos a la playa",                                                                                        # Phrase 4
           "Ich heiße Keyne, und das ist wunderbar",                                                                  # Phrase 5
           "she loves you much, mais elle te hait aussi and das ist traurig.", # Attention à cette phrase trilingue   # Phrase 6
           "A crane raises heavy construction materials.",                                                            # Phrase 7
           "Vogliamo visitare il Colosseo e nuotare nel Tevere.",                                                     # Phrase 8
           "私はそれについて全く知りません"                                                                              # Phrase 9
          ]
lang_exemples = ['deu','eng','fra','eng','spa','deu','en,fr,de','en','ita','jpn']

In [18]:
# Affichage des prédictions
print('Langue réelle                 :',lang_exemples)
print('Prédictions Naive Bayes       :',lang_id_nb(exemples))


Langue réelle                 : ['deu', 'eng', 'fra', 'eng', 'spa', 'deu', 'en,fr,de', 'en', 'ita', 'jpn']
Prédictions Naive Bayes       : ['German', 'English', 'French', 'English', 'Spanish', 'German', 'Galician', 'English', 'Italian', 'Japanese']


> **Recherche des phrases mal classées par Naive Bayes**

In [19]:
n_bad_max = 30
n_bad = 0
print("\tN°Ligne\tL. réelle\tPréd. Naive B.\t\tPhrase")
for i in range(len(df)):
    if (n_bad<n_bad_max):
        if (lan_to_language[df['lan_code'].iloc[i]] != lang_id_nb(df['sentence'].iloc[i])):
            n_bad +=1
            print(n_bad,'\t',i,'\t-',lan_to_language[df['lan_code'].iloc[i]][:12],'\t\t'+lang_id_nb(df['sentence'].iloc[i]).ljust(12)[:12],'\t\t'+
                  df['sentence'].iloc[i]," (proba={:.2f}".format(max(clf_nb.predict_proba(create_BOW([df['sentence'].iloc[i]]))[0]))+")")

	N°Ligne	L. réelle	Préd. Naive B.		Phrase
1 	 0 	- Berber langu 		Kabyle       		Yeqber uɛebbuḍ-iw seg wayen ččiɣ.  (proba=0.85)
2 	 5 	- Hindi 		Marathi      		बॅटर आऊट हो गया था।  (proba=0.85)
3 	 14 	- Turkish 		Kabyle       		Hadi yavrum kemik.  (proba=0.61)
4 	 33 	- Serbian 		Croatian     		Momci su predvidljivi.  (proba=0.68)
5 	 49 	- Swabian 		German       		Die Fraind hoißet sich aufrichdig, die Faind send's.  (proba=1.00)
6 	 55 	- Slovak 		Hungarian    		Hluk mi vadí.  (proba=0.90)
7 	 75 	- Berber langu 		Kabyle       		Bɣiɣ ad lemdeɣ ad cnuɣ am kemmini.  (proba=0.84)
8 	 97 	- Norwegian Ny 		Norwegian Bo 		Kvar er utgangen?  (proba=0.56)
9 	 111 	- Kabyle 		Berber langu 		Tom ad iṛuḥ ɣer Ustṛalya.  (proba=0.56)
10 	 138 	- Crimean Tata 		Berber langu 		Ötmek taze degil.  (proba=0.86)
11 	 140 	- Ukrainian 		Russian      		Я правий?  (proba=0.54)
12 	 184 	- Spanish 		Portuguese   		Malos pastores, malas ovejas.  (proba=0.56)
13 	 192 	- Slovenian 		Croatian     		Moje ime

In [20]:
from transformers import pipeline
import time

y_ext_predicted = []
y_ext_actual = []
y_nb_predicted=[]
dict_xlmr  = {"ar":"ara", "bg":"bul", "de":"deu", "el": "ell", "en":"eng", "es":"spa", "fr":"fra", "hi": "hin","it":"ita","ja":"jpn", \
              "nl":"nld", "pl":"pol", "pt":"por", "ru":"rus", "sw":"swh", "th":"tha", "tr":"tur", "ur": "urd", "vi":"vie", "zh":"cmn"}
lang_available = list(dict_xlmr.values())
lang_id_model_ext = pipeline('text-classification',model="papluca/xlm-roberta-base-language-detection")
start_time = time.time()
j= 0
for i in range(len(df_test)):
    if df_test.lan_code.iloc[i] in lang_available:
        y_ext_predicted.append(dict_xlmr[lang_id_model_ext(df_test.sentence.iloc[i][:1000])[0]['label']])
        y_ext_actual.append(df_test.lan_code.iloc[i])
        y_nb_predicted.append(y_pred[i])
        if (i-j)>=10000:
            j = (i//10000)*10000
            d = (time.time()- start_time)
            print("no",j," - ",d,"s (",d/len(y_ext_predicted),"s/id )          ",end="\r")
            
end_time = time.time()
duration = end_time - start_time
print("")
print("Nombre de phrases prises en compte :",len(y_ext_predicted))
print("Durée de traitement :",duration,"secondes, c'est à dire ", duration/len(y_ext_predicted),"s/identification")
print("Accuracy de xlm-roberta :",accuracy_score(y_ext_actual, y_ext_predicted))
print("Accuracy de Naïve Bayes :",accuracy_score(y_ext_actual, y_nb_predicted))

no 510000  -  9040.251601696014 s ( 0.027617145376077665 s/id )           
Nombre de phrases prises en compte : 329683
Durée de traitement : 9098.671464204788 secondes, c'est à dire  0.027598242748958206 s/identification
Accuracy de xlm-roberta : 0.9775056645322932
Accuracy de Naïve Bayes : 0.9932419930660665
